In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(1, '/gpfs/fs2/scratch/awhite38_lab/cfdsr/alcfd')
import utilities as utils

Defining input parameters

In [2]:
# note the unit for inlet_D
inlet_D = np.array([200]) # [um]
expansion_angle = np.array([40]) # [degrees]
inlet_v = np.array([0.5]) # [m/s]
inputs = [inlet_D, expansion_angle, inlet_v]

In [3]:
# Using blood as fluid
rho = 1060 # [kg m^-3]
muo = 0.004 # [Pa s]
re_number = utils.check_laminar(inlet_D*1e-6, inlet_v, rho, muo)
print(re_number)

(array([26.5]), array([], dtype=int64))


In [4]:
outputs = utils.run_model_expansion(*inputs, name='par_test', debug=True, run_parallel=True)
outputs

Inputs to the model are:
     D  Angle  V_in                                       outputs_file
0  200     40   0.5  /gpfs/fs2/scratch/awhite38_lab/cfdsr/alcfd/exa...
Model inputs are saved in:
 /gpfs/fs2/scratch/awhite38_lab/cfdsr/alcfd/examples/inputs_par_test.txt
Results are stored in:
 /gpfs/fs2/scratch/awhite38_lab/cfdsr/alcfd/examples/outputs_par_test.txt.


,Inlet D (um),Expansion angle,Mesh size,Viscosity,Density,Inlet P,Inlet V,System vol,Backflow percent,Backflow vol
0,200.0,40.0,0.000025,0.004,1060.0,15998.7,0.5,3.528010e-10,48.895581,1.725041e-10


In [ ]:
# /scratch/dfoster_lab/ansys2020R2/v202/Framework/bin/Linux64/runwb2 -B -R /scratch/awhite38_lab/cfdsr/alcfd/archived_models/expansion/script.wbjn